In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import os
import pandas as pd
import pickle
from tqdm import tqdm
from torch.utils.data import ConcatDataset

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [7]:
from modules.deep.DEEPNN import get_simple_deep_nn


def get_model(n_classes):
    return get_simple_deep_nn(n_classes)

In [ ]:
from modules.loaders.balanced_sampling import create_train_dataloader
from modules.loaders.resnet_features.features_dataset import FeaturesDataset
from modules.deep.DEEPNN import fix_batch_normalization_layers


def train_initial_NN(feats_S, lbls_S, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    assert (lbls_S.unique() == torch.arange(num_src_classes)).all()
    model = get_model(num_src_classes).cuda().train()
    model.apply(fix_batch_normalization_layers)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats_S, lbls_S)
    dl = create_train_dataloader(ds, 32, balanced)
    print('CLASSES:', num_src_classes)
    for ep in (pbar := tqdm(range(num_epochs))):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            preds = model(f.cuda())
            loss = loss_fn(preds, l.cuda())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        pbar.set_description('Loss: ' + str(avg_loss / len(dl)))
    return model.eval()

In [10]:
import numpy as np
from torch.utils.data import Subset


def train_NN(feats, lbls, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    feats, lbls = Subset(feats, np.arange(0, len(lbls))[lbls >= 0]), lbls[lbls >= 0]
    feats, lbls = Subset(feats, np.arange(0, len(lbls))[lbls < num_src_classes]), lbls[lbls < num_src_classes]
    model = get_model(num_src_classes).cuda().train()
    model.apply(fix_batch_normalization_layers)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats, lbls)
    dl = create_train_dataloader(ds, 32, balanced)
    for ep in (pbar := tqdm(range(num_epochs))):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        pbar.set_description('Loss: ' + str(avg_loss / len(dl)))
    return model.eval()

In [11]:
def predict_NN(model, feats, lbls):
    ds = FeaturesDataset(feats, lbls)
    dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for f,l in dl:
            out = model(f.cuda())
            preds.append(F.softmax(out, dim=1).detach().cpu())
    preds = torch.cat(preds, dim=0)
    return preds

In [1]:
from modules.logging.format_utils import format_measures
from modules.loaders.osda import create_datasets_sub
from modules.algorithms.base.OSLPP import Params, select_closed_set_pseudo_labels, evaluate_T
from modules.loaders.common import set_seed
from modules.selection.uncertanties import SelectRejectMode, select_initial_rejected, get_new_rejected

results = pd.DataFrame({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
for (source, target), (common, tgt_private) in config.items():
    for epochs in [35]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
                for seed in range(1):
                    set_seed(seed)
                    print(source, '->', target, 'lr=', lr, 'seed=', seed)
                    params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                              dataset='DomainNet_DCC', source=source, target=target,
                              num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                    (feats_S, labels_S), (feats_T, labels_T) = create_datasets_sub(params.dataset,
                                                                                   params.source,
                                                                                   params.target,
                                                                                   common,
                                                                                   tgt_private,
                                                                                   is_images=True)
                    params.n_r = int(len(labels_T) * n_r)
                    num_src_classes = params.num_common + params.num_src_priv

                    # initial
                    labels_S, labels_T = torch.tensor(labels_S), torch.tensor(labels_T)
                    feats_all = ConcatDataset([feats_S, feats_T])

                    t = 1
                    model = train_initial_NN(feats_S, labels_S, epochs, params, balanced=True, lr=lr)
                    predictions_S_2 = predict_NN(model, feats_S, labels_S)
                    predictions_T_2 = predict_NN(model, feats_T, labels_T)

                    confs, cs_pseudo_labels = predictions_T_2.max(dim=1)
                    selected = torch.tensor(select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), predictions_T_2,
                                                                            t, params.T,
                                                                            mode=select_reject_mode))
                    rejected = torch.tensor(select_initial_rejected(confs, predictions_T_2, params.n_r, mode=select_reject_mode))

                    rejected = torch.tensor(rejected)
                    selected = selected * (1 - rejected)

                    pseudo_labels = cs_pseudo_labels.clone()
                    pseudo_labels[rejected == 1] = num_src_classes
                    pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv,
                                         labels_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                    where = torch.where((selected == 1) + (rejected == 1))[0]
                    metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, labels_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                    print('______')
                    print(f'Iteration t={t}')
                    print('all: ', format_measures(metrics))
                    print('selected: ', format_measures(metrics_selected))
                    
                    for t in range(2, params.T):
                        print('TRAIN NN:', t)
                        model = train_NN(feats_all, torch.cat((labels_S, pseudo_labels), axis=0), epochs, params, balanced=True, lr=lr)
                        predictions_S_2 = predict_NN(model, feats_S, labels_S)
                        predictions_T_2 = predict_NN(model, feats_T, labels_T)

                        confs, cs_pseudo_labels = predictions_T_2.max(dim=1)
                        selected = torch.tensor(select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, predictions_T_2, select_reject_mode))
                        selected = selected * (1 - rejected)

                        rejected_new = get_new_rejected(confs, predictions_T_2, selected, rejected, mode=select_reject_mode)

                        rejected[rejected_new == 1] = 1
                        selected = selected * (1 - rejected)

                        pseudo_labels = cs_pseudo_labels.clone()
                        pseudo_labels[rejected == 1] = num_src_classes
                        pseudo_labels[(rejected == 0) * (selected == 0)] = -1
                        
                        print('Rejected:', len(rejected[rejected == 1]), '/', len(rejected))
                        print('Selected:', len(selected[selected == 1]), '/', len(selected))

                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, labels_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                        where = torch.where((selected == 1) + (rejected == 1))[0]
                        metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, labels_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                        print('______')
                        print(f'Iteration t={t}')
                        print('all: ', format_measures(metrics))
                        print('selected: ', format_measures(metrics_selected))

                    print((pseudo_labels == -1).sum())

                    _rejected = rejected
                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, labels_T.numpy(), cs_pseudo_labels.numpy(), _rejected.numpy())
                    print('all: ', format_measures(metrics))

                    results = results.append({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs}, ignore_index=True)

NameError: name 'pd' is not defined

In [ ]:
results.to_csv('results/abacaba-deep.csv', header=True, index=False)